In [ ]:
from openmm.app import *
from openmm import *
from sys import stdout
import MDAnalysis as mda
import os

md_dir = os.path.join("path to md dir"
                     )

# 读取 gro 文件
input_PDB = os.path.join(md_dir, "outsys.gro")
input_gro = os.path.join(md_dir, "outsys_new.gro")

u = mda.Universe(input_PDB, format="pdb")
u.atoms.write(input_gro)

In [ ]:
import MDAnalysis as mda

input_xtc = os.path.join(md_dir, "output.xtc")
pro_nucle_gro = os.path.join(md_dir, "PD.gro")
pro_nucle_xtc = os.path.join(md_dir, "PD.xtc")

# 读取 gro 和 xtc 文件
u = mda.Universe(input_gro, input_xtc)

# 选择蛋白质和核酸
protein_and_nucleic = u.select_atoms('protein or nucleic')
protein_and_nucleic.dimensions = u.dimensions
protein_and_nucleic.write(pro_nucle_gro)

# 保存新的 xtc 轨迹
with mda.Writer(pro_nucle_xtc, protein_and_nucleic.atoms.n_atoms) as W:
    for ts in u.trajectory[::100]:
        W.write(protein_and_nucleic.atoms)

print("新的 gro 和 xtc 文件已保存为 'PD.gro' 和 'PD.xtc'")

In [ ]:
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.transformations import unwrap, center_in_box
from MDAnalysis.analysis import align
from MDAnalysis import transformations


u = mda.Universe(pro_nucle_gro, pro_nucle_xtc)

pro_nucle_gro_fit = os.path.join(md_dir, "PD_fit.gro")
pro_nucle_xtc_fit = os.path.join(md_dir, "PD_fit.xtc")


# Select the protein and DNA molecules from the trajectory
protein = u.select_atoms('protein')
dna = u.select_atoms('nucleic')

# Select the protein-DNA complex (both protein and DNA)
complex = u.select_atoms('protein or nucleic')

# Guess and add bonds between atoms in the system
u.atoms.guess_bonds()

# Create a copy of the trajectory as a reference
ref_u = u.copy()

# Select the DNA molecule from the reference trajectory
reference = ref_u.select_atoms("protein")

# Define a workflow of transformations to preprocess the trajectory:
workflow = (
    # Step 1: Unwrap the complex to avoid broken molecules due to periodic boundaries
    transformations.unwrap(complex),
    
    # Step 2: Center the DNA molecule in the simulation box, using its mass center
    transformations.center_in_box(dna, center='mass'),
    
    # Step 3: Wrap the complex back into the simulation box, treating fragments as units
    transformations.wrap(complex, compound='fragments'),
    
    # Step 4: Align the DNA molecule to the reference DNA using rotation and translation
    transformations.fit_rot_trans(protein, reference)
)

# Apply the defined transformations to the trajectory
u.trajectory.add_transformations(*workflow)


# write new gro and xtc for fit&rot

u.atoms.write(pro_nucle_gro_fit)

with mda.Writer(pro_nucle_xtc_fit, u.atoms.n_atoms) as W:
    for ts in u.trajectory:
        W.write(u.atoms)


In [ ]:
# Visualize the processed trajectory using nglview
view = nv.show_mdanalysis(u)
view

In [ ]:
from MDAnalysis.analysis.rms import RMSD

# 选择当前轨迹（处理后的）中要计算RMSD的原子组，这里选择整个复合物
mobile_atoms = u.select_atoms('nucleic')

# 选择参考轨迹中对应的原子组，同样是整个复合物
reference_atoms = ref_u.select_atoms('nucleic')

# 创建RMSD分析对象
rmsd_analysis = RMSD(mobile_atoms, reference_atoms,
                     center=True,  # 是否将原子组移动到质心
                     superposition=True)  # 是否进行叠加（对齐），此处已对齐过但保持开启也可

# 将参考结构的帧（一般是第一帧）设置为参考帧
rmsd_analysis.run()

import matplotlib.pyplot as plt

rmsd_values = rmsd_analysis.rmsd[:, 2]  # 这里取第三列，具体根据RMSD结果数组结构来定

# 获取时间相关信息（假设时间步长已知，这里简单示例为帧索引作为时间单位，实际可根据轨迹的时间信息调整）
time_steps = range(len(rmsd_values))

# 绘制RMSD随时间变化的图
plt.plot(time_steps, rmsd_values)
plt.xlabel('Time Step')
plt.ylabel('RMSD (Å)')
plt.title('RMSD of DNA Over Time')
plt.show()

In [ ]:
import os, shutil
md_dir = os.path.join("ori_md_dir"
                     )
out_dir = os.path.join("save to a new dir"
                     )

def makedir(dirpath):
    if os.path.exists(dirpath) == 0:
        os.mkdir(dirpath)
    else:
        print(f'{dirpath}  already exist')
makedir(out_dir)

mdlist = os.listdir(md_dir)
for mdfile in mdlist:
    outmdfile = os.path.join(out_dir, mdfile)
    if "3hxo" not in mdfile:
        continue
    makedir(outmdfile)
    mdgro = os.path.join(md_dir, mdfile, 'PD_fit.gro')
    outmdgro = os.path.join(outmdfile, 'PD_fit.gro')
    mdxtc = os.path.join(md_dir, mdfile, 'PD_fit.xtc')
    outxtc = os.path.join(outmdfile, 'PD_fit.xtc')
    
    shutil.copy(mdgro, outmdgro)
    shutil.copy(mdxtc, outxtc)
        